<a href="https://colab.research.google.com/github/ghassen-elkamel/ghassen-elkamel/blob/main/whats_my_dinner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### connecting to kaggle to import the dataset

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install kaggle

In [ ]:
%cd /content/gdrive/MyDrive/AI/

/content/gdrive/MyDrive/AI


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/AI/"

## downloading the dataset

In [ ]:
!kaggle datasets download -d paultimothymooney/recipenlg

recipenlg.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip recipenlg.zip

Archive:  recipenlg.zip
replace RecipeNLG_code/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## reading the dataset

In [ ]:
import pandas as pd 
import numpy as np
import re
import nltk
dataset= pd.read_csv('RecipeNLG_dataset.csv',on_bad_lines='skip')

dataset[['title','directions','NER']].head(100)

,title,directions,NER
0,No-Bake Nut Cookies,"[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...
95,Baked Beans,"[""Cook onions and bell pepper in oil until oni...","[""pork"", ""bell pepper"", ""onions"", ""catsup"", ""b..."
96,Sweet-N-Sour Chicken,"[""Saute onion in 2 tablespoons shortening."", ""...","[""chicken"", ""shortening"", ""onion"", ""carrot chu..."
97,Ambrosia,"[""Combine all ingredients. Chill overnight."", ...","[""oranges"", ""moist coconut"", ""pecans"", ""cherri..."
98,Crazy Peanut Butter Cookies,"[""Mix together and roll into balls; flatten wi...","[""peanut butter"", ""sugar"", ""egg""]"


## cleaning the data

In [ ]:
import string
def remove_penctuation(txt):
    txt_nopenct="".join([c for c in txt if c not in string.punctuation])
    return  txt_nopenct

In [ ]:
dataset['NER_clean']=dataset['NER'].apply(lambda x: remove_penctuation(x))

## vectorizing the data 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
dataset_sample=dataset[0:]
tfidf=TfidfVectorizer()
x=tfidf.fit_transform(dataset_sample['NER_clean'])
print(x.shape)
print(x)

(2231142, 36338)
  (0, 2745)	0.40329676959072325
  (0, 27150)	0.21667613215634746
  (0, 29418)	0.3053836569763752
  (0, 29639)	0.503155010636841
  (0, 2779)	0.5267590380302817
  (0, 4396)	0.12860043267311236
  (0, 21973)	0.23654738685213772
  (0, 34324)	0.16066690925943194
  (0, 20246)	0.14767344304972646
  (0, 31375)	0.11009412196581282
  (0, 4017)	0.18511938810613796
  (1, 30261)	0.3230924059814809
  (1, 30242)	0.33277396816490434
  (1, 21116)	0.3853110509632457
  (1, 22085)	0.34121451471578984
  (1, 9085)	0.43793949339446214
  (1, 3783)	0.3983031433127904
  (1, 6616)	0.26228840454279123
  (1, 2329)	0.3174101940024895
  (2, 24363)	0.26913049723521376
  (2, 28023)	0.19700571139886225
  (2, 25621)	0.3444027942077792
  (2, 13271)	0.29030632609718554
  (2, 6375)	0.2777020101989687
  (2, 8636)	0.4517352760220029
  :	:
  (2231140, 9085)	0.06474459143366004
  (2231140, 27150)	0.0969321097685876
  (2231140, 4396)	0.0575306155417396
  (2231140, 31375)	0.04925164303545012
  (2231141, 24207)	0.

In [ ]:
from ctypes import sizeof
from numpy.ma.core import transpose
df=pd.DataFrame(x.toarray(),columns=tfidf.get_feature_names_out())
d=tfidf.get_feature_names_out()

df.head()
arr=df.to_numpy()

#g=tfidf.get_feature_names_out()[13]
print(arr)





[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## testing 

In [ ]:
def get_recipes(f,arr):

  c=0
  q=tfidf.get_feature_names_out()
  print (q)
  for i in range(1717):
    if q[i]==f :
     c=i

  t = []
  d={}
  a=0
  for j in range (100):
   v=arr[j][c]
   if v>0.0:
      #print(j)
    t.append (j)
    d[v]=j
    a=a+1
  t.sort(reverse=True)
 # print(d)
  for i in range(a):
    key=t[i]
    print("title:")
    print((dataset_sample['title'][key]))
    print("\n")
    print("gradients:\n")
    print((dataset_sample['NER_clean'][[key]]))
    print("steps:\n")
    print((dataset_sample['directions'][[key]]))
    




In [ ]:
element=input("give your element please: \n")
get_recipes(element,arr)

## saving the necessairy files to run our model

In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
from numpy import asarray
from numpy import savetxt
savetxt('data.csv', arr, delimiter=',')


In [ ]:
import numpy as geek
q=tfidf.get_feature_names_out()


q1=geek.array_str(q)

type(q1)
text_file = open("gradients_data.txt", "w")
 
text_file.write(q1)
 

text_file.close()


In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
q1=geek.array_str(q)

text_file = open("gradients_data.txt", "w")
q1=remove_penctuation(q1) 
print(q1)

text_file.write(q1)
 
text_file.close()


In [ ]:
xd = pd.DataFrame(dataset_sample, columns= ['title','directions','NER'])
df.to_csv (r'data2.csv', index = False, header=True)